In [24]:
import numpy as np
import torch
from scipy.stats import multivariate_normal
from OGMM import OGMM, init_cov_and_T_split

In [25]:
dataset = np.load('MGR.npy')
domain_names = ['Noiseless', "buccaneer2", "destroyerengine", "f16", "factory2"]

In [26]:
data_noiseless = dataset[:1000, :]
# feature column 17 is only zeros
data_noiseless = np.delete(data_noiseless, 17, axis=1)

# separation between training and validation data
data_noiseless = np.random.permutation(data_noiseless)
prop_train = 0.8
data_train = data_noiseless[:int(prop_train * len(data_noiseless)), :]
data_test = data_noiseless[int(prop_train * len(data_noiseless)):, :]


# Features of the noiseless data
X_train = data_train[:, :-2]
X_test = data_test[:, :-2]

# Labels of the noiseless data
y_train = data_train[:, -2]
y_test = data_test[:, -2]

Log likelihood function

In [27]:
from sklearn.mixture import GaussianMixture

GMM_sklearn = GaussianMixture(n_components=10, random_state=0).fit(X_train)

In [28]:
np.sum(GMM_sklearn.score_samples(X_test))

-42756.01079834077

In [29]:
def normal_pdf(x, mean, cov):
    return (2*np.pi)**(-len(x)/2) * np.linalg.det(cov)**(-1/2) * np.exp(-1/2 * (x - mean).T @ np.linalg.inv(cov) @ (x - mean))

In [30]:
def log_likelihood(mixture, X):
    pdf = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        for gauss in mixture:
            pdf[i] += gauss[0] * normal_pdf(X[i], gauss[2], gauss[3])
        if pdf[i] != 0:
            pdf[i] = np.log(pdf[i])
    return np.sum(pdf)

In [31]:
means = GMM_sklearn.means_
weights = GMM_sklearn.weights_
cov = GMM_sklearn.covariances_
GMM_sklearn_list = [[weights[i], 1, means[i], cov[i]] for i in range(2)]

In [32]:
log_likelihood(GMM_sklearn_list, X_test)

-47764.73041575383

In [33]:
range_data = np.mean(np.max(X_train, axis=0) - np.min(X_train, axis=0))

K_max = 10
cov_init, T_split = init_cov_and_T_split(K_max, range_data, X_train.shape[1])
X_train_t = torch.from_numpy(X_train)
GMM_online = OGMM(X_train_t, K_max, cov_init, 0.1, T_split)

C:\Users\SL276123\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\linalg\linalg.py:2140: RuntimeWarning: overflow encountered in cast
  r = r.astype(result_t, copy=False)
C:\Users\SL276123\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\linalg\linalg.py:2139: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [34]:
GMM_online

[[tensor(nan),
  tensor(nan),
  tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=torch.float64),
  tensor([[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]], dtype=torch.float64)],
 [tensor(nan),
  tensor(nan),
  tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, 